In [1]:
import pandas as pd
import numpy as np 
import pickle
from basic_analysis_toolkit import BasicAnalysis
from company_toolkit import *
import datetime


In [3]:
sp500_dataframe = pickle.load(open('stock_dataframe.pickle', 'rb'))        
company_dict = pickle.load(open('company_dictionary.pickle', 'rb'))
analysis = BasicAnalysis(sp500_dataframe)
start = datetime.date(day=1, month=9, year=2018)
end = datetime.date(day=1, month=9, year=2019)

In [4]:
# Factors I want to include:
company_ticker = 'AMZN'
company = company_dict.get(company_ticker)
# Industry dummy variables
# size
size = company.employees()
# price to book 
price_book = company.price_book()
# earning_yield
earnings_yield = 1/company.price_earnings()
# dividend yield
dividend_yield = company.forward_yield()
# momentum 
momentum = analysis.momentum(company_ticker, start, end, 10).mean()
# growth 
growth = get_growth(company.income['Gross Profit']).mean()
# earning variability
earning_variability = company.earnings_variability()
# financial leverage
fin_leverage_ratio = company.debt_equity()[-1]
# volatility
volatility = (analysis.get_close(company_ticker, start, end, 'B').var() ** .5)/analysis.get_close(company_ticker, start, end, 'B').mean()
# trading activity 
volume = analysis.get_trading_volume(company_ticker, start, end, 'BM').mean()

In [5]:
# creating the dataframe of all the information
# strategy: add each column individually, that way I can track and fix errors as they occur 
# creating size column
sizes = []
tickers = list(company_dict.keys())
list_ignored_companies = []
idx = []
for ticker in tickers:
    try:
        company = company_dict.get(ticker)
        size = company.employees()
        sizes.append(size)
        idx.append(ticker)
    except ValueError:
        list_ignored_companies.append(ticker)
sizes_series = pd.Series(sizes, index = idx)

In [6]:
# creating price to book column 
pb_ratios = []
tickers = sizes_series.index
idx = []
for ticker in tickers:
    try:
        company = company_dict.get(ticker)
        ratio = company.price_book().mean()
        pb_ratios.append(ratio)
        idx.append(ticker)
    except IndexError:
        list_ignored_companies.append(ticker)
pb_series = pd.Series(pb_ratios, index = idx)

In [7]:
# earnings yield column 
earnings_yield = []
tickers = pb_series.index
idx = []
for ticker in tickers:
    company = company_dict.get(ticker)
    y = 1/company.price_earnings()
    earnings_yield.append(y)
    idx.append(ticker)
earnings_series = pd.Series(earnings_yield, index = idx)

In [8]:
# dividend yield column
dividend_yields = []
tickers = pb_series.index
idx = []
for ticker in tickers:
    company = company_dict.get(ticker)
    dividend_yield = company.forward_yield()
    dividend_yields.append(dividend_yield)
    idx.append(ticker)
dividend_series = pd.Series(dividend_yields, index = idx)

In [9]:
# momentum column 
momentums = []
tickers = pb_series.index
idx = []
for ticker in tickers:
    momentum = analysis.momentum(ticker, start, end, 10).mean()
    idx.append(ticker)
    momentums.append(momentum)
momentum_series = pd.Series(momentums, index = idx)


In [10]:
# growth column
growths = []
idx = []
tickers = pb_series.index
for ticker in tickers:
    company = company_dict.get(ticker)
    growth = get_growth(company.income['Gross Profit']).mean()
    growths.append(growth)
    idx.append(ticker)
growth_series = pd.Series(growths, index=idx)

In [11]:
# earning variability column
earning_vars = []
idx = []
tickers = pb_series.index
for ticker in tickers:
    company = company_dict.get(ticker)
    earning_variability = company.earnings_variability()
    earning_vars.append(earning_variability)
    idx.append(ticker)
earning_var_series = pd.Series(earning_vars, index = idx)


In [12]:
# financial leverage
values = []
idx = []
tickers = pb_series.index
for ticker in tickers:
    company = company_dict.get(ticker)
    fin_leverage_ratio = company.debt_equity().mean()
    values.append(fin_leverage_ratio)
    idx.append(ticker)
fin_leverage_series = pd.Series(values, index=idx)

In [13]:
values = []
idx = []
tickers = pb_series.index
# volatility
for ticker in tickers:
    volatility = (analysis.get_close(ticker, start, end, 'B').var() ** .5)/analysis.get_close(company_ticker, start, end, 'B').mean()
    idx.append(ticker)
    values.append(volatility)
volatility_series = pd.Series(values, index=idx)

In [14]:
# trading activity 
values = []
idx = []
tickers = pb_series.index
for ticker in tickers:
    volume = analysis.get_trading_volume(ticker, start, end, 'BM').mean()
    values.append(volume)
    idx.append(ticker)
volume_series = pd.Series(values, index=idx)

In [15]:
fundemental_values = pd.DataFrame(index = pb_series.index)

In [16]:
fundemental_values['Size (num of employees)'] = sizes_series

In [17]:
fundemental_values['Price to Book'] = pb_series

In [18]:
fundemental_values['Earnings Yield'] = earnings_series

In [19]:
fundemental_values['Dividend Yield'] = dividend_series

In [20]:
fundemental_values['Momentum'] = momentum_series

In [21]:
fundemental_values['Growth'] = growth_series

In [22]:
fundemental_values['Earning Variability'] = earning_var_series

In [23]:
fundemental_values['Financial Leverage'] = fin_leverage_series

In [24]:
fundemental_values['Volatility'] = volatility_series

In [25]:
fundemental_values['Trading Volume'] = volume_series

In [26]:
fundemental_values = fundemental_values.fillna(value=0)

In [27]:
fundemental_values.sort_values(by=['Price to Book'], ascending=False)

,Size (num of employees),Price to Book,Earnings Yield,Dividend Yield,Momentum,Growth (gross profit increase),Earning Variability,Financial Leverage,Volatility,Trading Volume
FOXA,7700.0,inf,0.074460,1.39,-0.21048,0.071313,0.303118,0.456431,0.000851,6.130167e+06
DOW,37600.0,inf,0.079239,6.57,-0.29476,0.063643,1.139710,1.073300,0.001642,4.569183e+06
FOX,7700.0,inf,0.075301,1.40,-0.26996,0.071313,0.303118,0.456431,0.001107,2.378933e+06
MKTX,480.0,29.494703,0.011692,0.51,8.16216,0.144187,0.240434,0.126049,0.034240,8.887417e+05
CBOE,842.0,16.911813,0.038139,1.21,0.63892,0.329471,0.413153,0.384873,0.004365,9.556000e+05
...,...,...,...,...,...,...,...,...,...,...
ABMD,1371.0,0.000000,0.029087,0.00,-8.04036,0.000000,0.000000,0.000000,0.033392,6.374417e+05
REGN,7549.0,0.000000,0.061767,0.00,-4.26904,0.000000,0.000000,0.000000,0.025261,8.144500e+05
SNPS,13245.0,0.000000,0.027816,0.00,1.35272,0.000000,0.000000,0.000000,0.009834,1.483092e+06
ROP,15611.0,0.000000,0.028645,0.49,2.12736,0.000000,0.000000,0.000000,0.020482,6.020000e+05


In [28]:
pickle.dump(fundemental_values, open('fundemental_risk_model_data.pickle', 'wb'))

In [29]:

df = fundemental_values.fillna(value=0)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(value=0)
df_norm = (df - df.mean()) / (df.max() - df.min())


In [30]:
pickle.dump(df_norm, open('fundemental_risk_model_normdata.pickle', 'wb'))

In [31]:
df_norm

,Size (num of employees),Price to Book,Earnings Yield,Dividend Yield,Momentum,Growth (gross profit increase),Earning Variability,Financial Leverage,Volatility,Trading Volume
MMM,0.018842,0.012940,-0.002029,0.139875,-0.095648,-0.012222,-0.010549,0.002548,0.045441,-0.011271
ABT,0.023387,0.015386,-0.002346,-0.064693,0.029631,0.016550,0.004781,0.000305,-0.034354,0.009159
ABBV,-0.009797,-0.022207,-0.002131,0.420908,-0.053604,0.004403,-0.010158,0.016171,-0.019684,0.018091
ABMD,-0.022811,-0.076226,-0.002245,-0.213650,-0.371132,-0.017673,-0.011539,-0.003120,0.325060,-0.021607
ACN,0.195674,0.129008,-0.002179,-0.067672,0.045627,-0.000135,-0.008642,0.000726,0.028568,-0.013480
...,...,...,...,...,...,...,...,...,...,...
XYL,-0.015707,0.001098,-0.002134,-0.091505,-0.008205,0.010581,-0.004514,0.001064,-0.037633,-0.018993
YUM,-0.007979,0.147122,-0.002182,-0.072637,0.050973,-0.015343,-0.009110,-0.001690,-0.012056,-0.012440
ZBH,-0.014797,-0.038900,-0.002536,-0.143144,0.020312,0.000054,0.030080,0.000734,-0.013437,-0.018263
ZION,-0.018740,-0.072400,-0.001524,0.115049,-0.025075,0.006769,-0.001142,0.023692,-0.053111,-0.011874


In [32]:
tickers = list(df_norm.index)
return_data = pd.DataFrame()
for ticker in tickers:
    data = analysis.get_percent_change(ticker, start, end, 'BM')
    return_data[ticker] = data

In [33]:
return_data

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
2018-09-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-10-31,-0.097053,-0.060251,-0.176887,-0.241356,-0.073913,-0.169972,-0.089609,-0.410489,-0.050912,0.041429,...,0.015751,-0.208248,0.038127,0.032987,0.064862,-0.178916,-0.005500,-0.136001,-0.061815,-0.015400
2018-11-30,0.092820,0.074122,0.210918,-0.024971,0.043776,-0.277625,0.020874,0.169687,0.112356,0.062414,...,0.113789,0.087475,0.070190,-0.034087,0.083285,0.112839,0.020020,0.030196,0.034219,0.041265
2018-12-31,-0.083590,-0.023228,-0.022064,-0.022965,-0.142901,-0.066359,-0.098250,-0.133333,-0.113950,-0.066494,...,-0.047601,-0.095887,-0.060629,-0.265973,-0.079044,-0.085777,-0.003253,-0.113656,-0.162762,-0.088740
2019-01-31,0.051223,0.008987,-0.129081,0.080082,0.088930,0.014387,0.095385,0.322319,0.011050,0.133472,...,0.071974,0.243656,0.062716,0.427632,0.314313,0.068046,0.022411,0.056305,0.168139,0.007248
2019-02-28,0.035397,0.063579,-0.013078,-0.047199,0.050993,-0.107959,0.059237,-0.036051,0.016206,0.051251,...,0.056699,0.028697,0.047746,0.095356,0.119350,0.060202,0.005533,0.132895,0.073755,0.093663
2019-03-29,0.001881,0.029889,0.017037,-0.146218,0.090718,0.080446,0.015200,0.084573,0.054086,0.049333,...,0.021102,-0.057057,0.024608,0.034951,0.011891,0.046195,0.056190,0.028843,-0.111350,0.068343
2019-04-30,-0.087930,-0.004754,-0.014890,-0.028642,0.037780,0.058862,0.085407,0.082680,-0.024688,-0.053097,...,0.049473,0.210610,0.005159,0.043152,-0.052449,0.055162,0.045887,-0.035552,0.086325,0.011622
2019-05-31,-0.157037,-0.043112,-0.033757,-0.055838,-0.025182,-0.100394,-0.063440,-0.007962,-0.068062,-0.077103,...,-0.047955,-0.256975,0.014867,-0.082434,-0.148410,-0.110072,-0.019542,-0.074943,-0.126901,-0.007757
2019-06-28,0.085070,0.104689,-0.052014,-0.005460,0.037626,0.088310,0.087671,0.107990,-0.005548,0.060759,...,0.091396,0.155222,0.037496,0.156811,0.152576,0.126920,0.081290,0.033442,0.067564,0.123107
